In [1]:
# 不使用 __slots__ 的情況  
import sys  
from typing import NamedTuple, Optional  
  
class ActionWithoutSlots(NamedTuple):  
    type: str  
    payload: Optional[object] = None  
  
# 使用 __slots__ 的情況  
class ActionWithSlots:  
    __slots__ = ('type', 'payload')  
      
    def __init__(self, type: str, payload: Optional[object] = None):  
        self.type = type  
        self.payload = payload  
  
# 創建實例  
action1 = ActionWithoutSlots("increment")  
action2 = ActionWithSlots("increment")  
  
# 計算記憶體大小  
size1 = sys.getsizeof(action1)  
size2 = sys.getsizeof(action2)  
  
print(f"不使用 __slots__: {size1} bytes")  
print(f"使用 __slots__: {size2} bytes")  
print(f"記憶體節省: {size1 - size2} bytes ({(1 - size2/size1) * 100:.2f}%)")

不使用 __slots__: 56 bytes
使用 __slots__: 48 bytes
記憶體節省: 8 bytes (14.29%)


In [4]:
!pip install pystorex==0.1.7

  Attempting uninstall: pystorex
    Found existing installation: pystorex 0.1.6
    Uninstalling pystorex-0.1.6:
      Successfully uninstalled pystorex-0.1.6


In [ ]:
from pystorex import (
    create_store,
    create_reducer,
    on,
    create_effect,
    create_selector,
)
from pydantic import BaseModel
from pystorex.actions import create_action

# 1. 定义你的 State 模型
class CounterState(BaseModel):
    count: int = 0

# 2. 创建 Actions
increment = create_action("increment")
decrement = create_action("decrement")
reset = create_action("reset", lambda value: value)
increment_by = create_action("incrementBy", lambda amount: amount)

# 3. 创建 Reducer
def counter_handler(state: CounterState, action):
    new_state = state.model_copy()
    if action.type == increment.type:
        new_state.count += 1
    elif action.type == decrement.type:
        new_state.count -= 1
    return new_state

counter_reducer = create_reducer(
    CounterState(),
    on(increment, counter_handler),
    on(decrement, counter_handler),
)

# 4. 初始化 Store（不再传初始 state）
store = create_store()

#    register_root 内部会发出 `[Root] Init Store`，
#    并用每个 reducer.initial_state 填充 store.state
store.register_root({"counter": counter_reducer})

# 5. 订阅 state 变化
#    由于 register_root 时我们对 select() 做了 skip(1)，
#    这里不会收到那次空的初始注入。
store.select(lambda s: s["counter"].count).subscribe(
    lambda new: print("Count:", new[1])
)

# 6. Dispatch Actions
store.dispatch(increment())  # Count: 1
store.dispatch(increment())  # Count: 2
store.dispatch(decrement())  # Count: 1

Count: 1
Count: 2
Count: 1


Action(type='decrement', payload=None)

In [ ]:
import time
from typing import Optional
from pydantic import BaseModel
from reactivex import operators as ops

from pystorex.actions import create_action
from pystorex import create_store, create_reducer, on, create_effect
from pystorex.store_selectors import create_selector
from pystorex.middleware import LoggerMiddleware

# 1. 定義狀態模型
class CounterState(BaseModel):
    count: int = 0
    loading: bool = False
    error: Optional[str] = None
    last_updated: Optional[float] = None

# 2. 定義 Actions
increment = create_action("increment")
decrement = create_action("decrement")
reset = create_action("reset", lambda value: value)
increment_by = create_action("incrementBy", lambda amount: amount)

load_count_request = create_action("loadCountRequest")
load_count_success = create_action("loadCountSuccess", lambda value: value)
load_count_failure = create_action("loadCountFailure", lambda error: error)

# 3. 定義 Reducer
def counter_handler(state: CounterState, action) -> CounterState:
    new_state = state.copy(deep=True)
    now = time.time()

    if action.type == increment.type:
        new_state.count += 1
        new_state.last_updated = now
    elif action.type == decrement.type:
        new_state.count -= 1
        new_state.last_updated = now
    elif action.type == reset.type:
        new_state.count = action.payload
        new_state.last_updated = now
    elif action.type == increment_by.type:
        new_state.count += action.payload
        new_state.last_updated = now
    elif action.type == load_count_request.type:
        new_state.loading = True
        new_state.error = None
    elif action.type == load_count_success.type:
        new_state.loading = False
        new_state.count = action.payload
        new_state.last_updated = now
    elif action.type == load_count_failure.type:
        new_state.loading = False
        new_state.error = action.payload

    return new_state

counter_reducer = create_reducer(
    CounterState(),
    on(increment, counter_handler),
    on(decrement, counter_handler),
    on(reset, counter_handler),
    on(increment_by, counter_handler),
    on(load_count_request, counter_handler),
    on(load_count_success, counter_handler),
    on(load_count_failure, counter_handler),
)

# 4. 定義 Effects
class CounterEffects:
    @create_effect
    def load_count(self, action_stream):
        return action_stream.pipe(
            ops.filter(lambda action: action.type == load_count_request.type),
            ops.do_action(lambda _: print("Effect: Loading counter...")),
            ops.delay(1.0),
            ops.map(lambda _: load_count_success(42))
        )


# 5. 建立 Store、註冊模組
store = create_store()
store.apply_middleware(LoggerMiddleware)
store.register_root({"counter": counter_reducer})
store.register_effects(CounterEffects)

# 6. 訂閱狀態與測試
get_counter_state = lambda state: state["counter"]
get_count = create_selector(
    get_counter_state,
    result_fn=lambda counter: counter.count or 0
)
store.select(get_count).subscribe(
    lambda c: print(f"Count: {c[1]}")
)

# 7. 執行操作示例
if __name__ == "__main__":
    store.dispatch(increment())
    store.dispatch(increment_by(5))
    store.dispatch(decrement())
    store.dispatch(reset(10))
    store.dispatch(load_count_request())
    # 給 Effects 一些時間
    time.sleep(2)


▶️ dispatching increment
🔄 state before increment: {'counter': CounterState(count=0, loading=False, error=None, last_updated=None)}
Count: 1
✅ state after increment: {'counter': CounterState(count=1, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching incrementBy
🔄 state before incrementBy: {'counter': CounterState(count=1, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 6
✅ state after incrementBy: {'counter': CounterState(count=6, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching decrement
🔄 state before decrement: {'counter': CounterState(count=6, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 5
✅ state after decrement: {'counter': CounterState(count=5, loading=False, error=None, last_updated=1745050111.1340942)}
▶️ dispatching reset
🔄 state before reset: {'counter': CounterState(count=5, loading=False, error=None, last_updated=1745050111.1340942)}
Count: 10
✅ state after reset: {'counter': Co

: 